# Support Vector Machines

Support Vector Machines könnten zwar mit Stützvektor-Maschinen übersetzt werden, aber tatsächlich hat sich auch im deutschprachigen Raum der englische Fachbegriff eingebürgert. Allerdings wird meist nur die Abkürzung verwendet, also SVM.  

SVMs gehören zum überwachten maschinellem Lernen (Supervised Learning). Sie können sowohl für Klassifikations- also auch Regressionsprobleme eingesetzt werden. Nachdem wir aber uns zu letzt mit linearer und polynomialer Regression beschäftigt haben, führen wir die SVMs anhand von Klassifikationsproblemen ein.

Wie üblich laden wir einige Module, die wir brauchen. 

Hinweis: Die folgenden Code-Beispiels sind größtenteils dem Buch "Data Science mit Python"von Jake VanderPlas übernommen.


In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from scipy import stats

Genau wie in der letzten Woche möchten wir vorerst keine echten Daten importieren, sondern erzeugen uns künstliche Daten. Dazu nehmen wir Scikit-Learn zur Hilfe, das die Funktion ``make_blobs`` zur Verfügung stellt. Details dazu finden Sie in der Dokumentation hier: https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_blobs.html

In [ ]:
from sklearn.datasets import make_blobs

X, y = make_blobs(n_samples=50, centers=2,
                  random_state=0, cluster_std=0.60)
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn');

Die Messdaten haben zwei Attribute (Inputs) und ein Target (Output), das nur die Zugehörigkeit zu einer Klasse (gelb oder rot) kennzeichnet. Sie können auch in X und y hineinsehen: 

In [ ]:
print(X)
print(y)

Der Output y enthält entweder eine 0 oder eine 1. In der Grafik steht rot für 0 und gelb für 1.

### Ziel der Klassifikation mit SVM-Algorithmen
*Ziel: Wir möchten nun eine Gerade (oder später verallgemeinert eine Kurve) finden, die eindeutig die roten Bubbles von den gelben trennt.*

Es ist nicht so schwierig, sich lineare Funktionen oder Geraden auszudenken, die die roten und gelben Bubbles trennen.


**Mini-Übung**

Denken Sie sich eine Gerade aus, die rote und gelbe Bubbles trennt und zeichnen Sie diese ein.

In [ ]:
# Hier Ihr Code

Wenn wir unsere Geraden vergleichen würden, hat wahrscheinlich jeder von uns eine andere Gerade gewählt. Und damit sind wir wieder bei dem Problem: welches Modell ist das beste?

Im Folgenden sehen Sie einige Geraden eingezeichnet. Je nachdem, für welche Gerade wir uns entscheiden würden, würde das blaue Kreuz zu den roten oder zu den gelben Bubbles gehören. 

In [ ]:
xfit = np.linspace(-1, 3.5)
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn')
plt.plot([0.6], [2.1], 'x', color='blue', markeredgewidth=2, markersize=10)

for m, b in [(1, 0.65), (0.5, 1.6), (-0.2, 2.9)]:
    plt.plot(xfit, m * xfit + b, '-k')

plt.xlim(-1, 3.5);

### Idee SVM: maximiere den Rand

Die Idee der SVM-Algorithmen ist es, nicht nur eine Linie (Kurve) zu finden, die die einzelnen Datenpunkte voneinander trennt, sondern diejenige Linie (Kurve), de auch noch den breitesten Rand hat.

Wir zeichnen nun zu den drei Linien Ränder ein. Dabei wählen wir den Rand so breit, bis zum ersten Mal ein Datenpunkt getroffen wird.

In [ ]:
X, y = make_blobs(n_samples=50, centers=2,
                  random_state=0, cluster_std=0.60)

xfit = np.linspace(-1, 3.5)

fig, ax = plt.subplots(figsize=(12,8))
ax.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn')

for m, b, d in [(1, 0.65, 0.33), (0.5, 1.6, 0.55), (-0.2, 2.9, 0.2)]:
    yfit = m * xfit + b
    ax.plot(xfit, yfit, '-k')
    ax.fill_between(xfit, yfit - d, yfit + d, edgecolor='none', color='#AAAAAA', alpha=0.4)

ax.set_xlim(-1, 3.5);

Bei einer Klassifikation mit SVM wählen wir nun diejenige Linie, die den maximalen Rand zu den Datenpunkte hat, also die mittlere Linie. Ein SVM ist ein **Breiter-Rand-Klassifikator**.

### SVM mit Scikit-Learn bestimmen

Scikit-Learn enthält sowohl für die Regression als auch für die Klassifikation Support-Vector-Machines-Algorithmen. Das Untermodul für SVMs heißt ``sklearn.svm``. Da wir ein Klassifikationsproblem modellieren wollen, importieren wir einen SVM Classifier, kurz SVC. Die Dokumentation dazu finden Sie hier

> https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html

Danach wählen wir ein Modell aus. Wir entscheiden uns für eine lineare Funktion und setzen daher den ersten Hyperparameter ``kernel`` auf ``'linear'``. Zusätzlich müssen wir noch einen Hyperparameter setzen, der bestimmt, wie strikt wir bei Verletzungen der Ränder sind. Für den Anfang wollen wir den breitesten Rand, der möglich ist, und setzen daher den Hyperparameter ``C`` auf den Wert ``1E10``.

In [ ]:
from sklearn.svm import SVC # "Support vector classifier"

X, y = make_blobs(n_samples=50, centers=2, random_state=0, cluster_std=0.60)

# Auswahl Modell
model = SVC(kernel='linear', C=1E10)

# Training 
model.fit(X, y)

Um jetzt besser zu verstehen, was der SVM-Algorithmus gemacht hat, übernehmen wir von Jake VanderPlas eine Funktion, die uns die Ränder einzeichnet.

In [ ]:
def plot_svc_decision_function(model, ax=None, plot_support=True):
    """Plot the decision function for a 2D SVC"""
    if ax is None:
        ax = plt.gca()
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    
    # create grid to evaluate model
    x = np.linspace(xlim[0], xlim[1], 30)
    y = np.linspace(ylim[0], ylim[1], 30)
    Y, X = np.meshgrid(y, x)
    xy = np.vstack([X.ravel(), Y.ravel()]).T
    decision_function = model.decision_function(xy)
    if decision_function.ndim == 1:
        P = decision_function.reshape(X.shape)
        ax.contour(X, Y, P, colors='k', levels=[-1, 0, 1], alpha=0.5,linestyles=['--', '-', '--'])
    else:
        for i in range(np.shape(decision_function)[1]):
            P = decision_function[:,i].reshape(X.shape)
            # plot decision boundary and margins
            ax.contour(X, Y, P, colors='k', levels=[-1, 0, 1], alpha=0.5,linestyles=['--', '-', '--'])
    
    # plot support vectors
    if plot_support:
        ax.scatter(model.support_vectors_[:, 0], model.support_vectors_[:, 1],
                   s=300, linewidth=1, facecolors='none', edgecolors='black');
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

Mit dieser Funktion zeichen wir nun das Ergebnis des trainierten Modells zusammen mit den Rändern ein. Zusätzlich werden einige Punkte eingekreist.

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))
ax.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn')
plot_svc_decision_function(model);

Es gibt zwei rote Punkte und einen gelben Punkt, die direkt den Rand berühren. Die Ortsvektoren, die vom Koordinatenursprung zu diesen Punkten zeigen, heißen Stützvektoren, also auf Englisch **support vector** und haben dieser Methode zu ihrem Namen verholfen. 

Weil diese Support Vectors wichtig sind, werden Sie nach dem Training im Modell abgespeichert.

In [ ]:
model.support_vectors_

Support Vector Machines sind sehr beliebt. Einer der wichtigsten Gründe für ihre Beliebtheit ist, dass nur noch die Position der Stützvektoren entscheidet, wo die Datnepunkte getrennt werden . Das ist eine simple und vor allem auch leicht nachvollziehbare Entscheidung des Algorithmus. Bei vielen anderen ML-Algorithmen erkennt man nur schwer, was dazu geführt hat, dass dieses Modellergebnis herauskommt.

Ein zweiter Grund ist, dass jeder zusätzliche Punkt, der dazu kommt, die Stützvektoren nicht ändert, sofern der Punkt auf der richtigen Seite liegt. Damit lohnt es sich, SVMs auch mit Daten zu trainieren, wenn man weiß, dass später neue Messungen hinzukommen.

Hier ein Beispiel, bei dem wir uns ansehen, was das Modell durch die ersten 60 Punkte und was durch die ersten 120 Punkte gelernt hat:


In [ ]:
def plot_svm(N=10, ax=None):
    X, y = make_blobs(n_samples=200, centers=2,
                      random_state=0, cluster_std=0.60)
    X = X[:N]
    y = y[:N]
    model = SVC(kernel='linear', C=1E10)
    model.fit(X, y)
    
    ax = ax or plt.gca()
    ax.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn')
    ax.set_xlim(-1, 4)
    ax.set_ylim(-1, 6)
    plot_svc_decision_function(model, ax)

fig, ax = plt.subplots(1, 2, figsize=(16, 6))
fig.subplots_adjust(left=0.0625, right=0.95, wspace=0.1)
for axi, N in zip(ax, [60, 120]):
    plot_svm(N, axi)
    axi.set_title('N = {0}'.format(N))

Wir sehen, dass bereits die ersten 60 Punkte (linkes Bild) zu einem sehr guten Modell geführt haben. Acu wenn wir weitere 60 Punkte hinzunehmen (rechtes Bild zeigt die ersten 120 Trainingsdaten), bleibt die Gerade und auch dieselben Stützvektoren werden gefunden.

In [ ]:
#from ipywidgets import interact, fixed
#interact(plot_svm, N=np.arange(10,200,20), ax=fixed(None));

### SVM mit Kernel-Methoden kombiniert

Für die polynomiale Regression hat Sckit-Learn gar keine eigenständige Funktion. Stattdessen haben wir auf einen Trick zurückgegriffen, bei dem aus einem niedrig-dimensionalen Raum (wenige Eigenschaften, Attribute, Features) ein höher-dimensionaler Raum gemacht wurde: Stichwort Kernel-Trick! Auch hier wollen wir dieses Konzept nutzen.

Zuerst generieren wir uns wieder künstliche Daten. Scikit-Learn stellt uns eine Funktion zur Verfügung, mit der wir Trainingsdaten in Kreisen erzeugen können, nämlich ``make_circles`` aus dem Untermodul ``sklearn.datasets``. Weitere Details finden Sie in der Dokumentation: https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_circles.html?highlight=make%20circles#sklearn.datasets.make_circles 

In [ ]:
from sklearn.datasets import make_circles

# Erzeugung künstlicher Trainingsdaten
X, y = make_circles(100, factor=.1, noise=.1)

# Visualisierung
fig, ax = plt.subplots(figsize=(12,8))
ax.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn');


Keine einzige Gerade, die wir uns ausdenken, trennt die gelben von den roten Bubbles. Aber wir können eine Dimension, eine weitere Eigenschaft hinzunehmen. Die Position der Daten wird durch ihre x-Koordinate und ihre y-Koordinate beschrieben. Wir wollen jetzt noch eine z-Koordinate in die Höhe hinzufügen. Dabei soll die z-Koordinate umso höher sein, je n|aher der Datenpunkt am Ursprung (0,0) liegt. 

Bevor wir die Messdaten in eine höhere Dimension projizieren, basteln wir uns erstmal eine Funktion, die prinzipiell diese Gestalt hat. Im eindimensionalen gedacht soll an der Stelle 0 der Wert 1 (entspricht 100 %) erzielt werden und danach soll die Funktion schnell abfallen. Das erreichen wir beispielsweise mit einer Exponentialfunktion mit einem negativen Exponenten (Hochzahl).

In [ ]:
x = np.linspace(0, 5)
y = np.exp(-x)

fig, ax = plt.subplots()
ax.plot(x,y);

Jetzt brauchen wir aber eine Funktion, die von zwei Koordinaten abhängt, denn wir haben ja zwei Input-Features. Wir berechnen zu jeden Punkt den Abstand zum Ursprung (Wurzelziehen schenken wir uns). Dann sieht die Funktion folgendermaßen aus:

In [ ]:
#%matplotlib widget
from mpl_toolkits.mplot3d import Axes3D

# Gitter erzeugen
X_grid, Y_grid = np.meshgrid(np.linspace(-3, 3), np.linspace(-3, 3))

# Funktion auswerten und Höheninformationen berechnen
z = np.exp( -(X_grid**2 + Y_grid**2) )
           
# 3D Plot 
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(X_grid, Y_grid, z, cmap='viridis');


Jetzt nutzen wir genau diese Funktion, um die z-Koordinate für unsere Trainingsdaten zu berechnen.

In [ ]:
# Erzeugung künstlicher Trainingsdaten
X, y = make_circles(100, factor=.1, noise=.1)

# Kernel-Trick
z = np.exp( -(X[:,0]**2 + X[:,1]**2) )

# 3D Plot
fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X[:,0], X[:,1], z, s=30, c=y, cmap='autumn');

Wenn wir uns jetzt die 3D-Daten ansehen, können wir sehr wohl die gelben von den roten Bubbles unterscheiden: eine Ebene mit der Ebenengleichung $z = 0.7$ würde beispielsweise funktionieren.

Die vorgestellte Vorgehensweise hat einen Nachteil. Unsere Messdaten wurden künstlich erzeugt, indem zwei Kreise um den Ursprung erzeugt aurden und dann die Bublles ein wenig verrauscht und gestört wurden. Daher fiel es uns einfach zu raten, dass die Kernel-Methode mit einer Exponentialfunktion bezogen auf den Abstand der Punkte zum Ursprung gut funktionieren würde. Aber wie finden wir für nicht künstlich erzeugte Messdaten eine passende Kernel-Funktion? 

Tatsächlich wird in der Praxis für jeden einzelnen Datenpunkt eine eigene Kernel-Funktion genutzt, mit der in den höherdimensionalen Raum projiziert wird. Wenn wir dabei ganz bestimmte Kernel-Funktionen nehmen, ist die Berechnung auch nicht allzu aufwendig, weil wir Funktionseigenschaften geschickt ausnutzen können. 

### Finetuning des 1. Hyperparameters: 'linear' oder 'rbf'

In Scikit-Learn können wir die dort implementierten Algorithmen einfach nutzen, in dem wir den Hyperparameter ``'linear'`` auf ``'rbf'`` umstellen. Dabei steht rbf für radiale Basis-Funktion und meint die mehrdimensionalen Exponentialfunktionen, die vom Abstand abhängen.

In [ ]:
# Erzeugung künstlicher Trainingsdaten
X, y = make_circles(100, factor=.1, noise=.1)

# Auswahl Modell
model = SVC(kernel='rbf', C=1E10)

# Training 
model.fit(X, y)

# Visualisierung
fig, ax = plt.subplots(figsize=(8,6))
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn')
plot_svc_decision_function(model)


### Finetuning des 2. Hyperparameters: C

Bei den künstlich erzeugten Messdaten haben wir zwei verrauschte Kreise gewählt, die zwar nicht durch eine Gerade getrennt werden konnten, aber eindeutig durch eine Kurve. Was aber, wenn die Messdaten ein wenig überlappen?

In [ ]:
# erzeuge künstliche Daten, die überlappen
X, y = make_blobs(n_samples=100, centers=2, random_state=0, cluster_std=1.2)

# Visualisierung
fig, ax = plt.subplots(figsize=(8,6))
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn');

Für diesen Fall gibt es den 2. Hyperparameter, das ``'C'``, mit dem die Härte des Randes eingestellt wird. Eine große Zahl bedeutet, dass die Grenze absolt hart und unüberwindlich ist. Am besten sehen wir uns das für unsere Trainingsdaten an.

In [ ]:
# erzeuge künstliche Messdaten
X, y = make_blobs(n_samples=100, centers=2, random_state=0, cluster_std=0.8)

# zeichne Messdaten
fig, ax = plt.subplots(1, 2, figsize=(12, 8))
fig.subplots_adjust(left=0.0625, right=0.95, wspace=0.1)

# trainiere zwei SVM: C = 10 und C = 0.1 und zeichne die Ränder 
for axi, C in zip(ax, [10.0, 0.1]):
    model = SVC(kernel='linear', C=C).fit(X, y)
    axi.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn')
    plot_svc_decision_function(model, axi)
    axi.scatter(model.support_vectors_[:, 0],
                model.support_vectors_[:, 1],
                s=300, lw=1, facecolors='none');
    axi.set_title('C = {0:.1f}'.format(C), size=14)

Bei C = 10 ragt kein Messpunkt in den Randbereich. Aufgrund der Positionen der Messpunkte ist dafür der Randbereich sehr klein. Die rechte Grafik zeigt, dass einige rote und gelbe Bubbles im Randbereich liegen. 

Natürlich stellt sich als nächstes die Frage: welches C sollen wir wählen? Diese Frage wird wie zuvor bei der (linearen und polynomialen) Regression dadurch beantwortet, dass man die Daten in Trainings- und Testdaten aufteilt und die Prognosequalität misst. Einen R2-Score gibt es nur bei Regressionspolynomen. Für die Support Vector Machines wird einfach nur der Score berechnet. 

In [ ]:
from sklearn.model_selection import train_test_split

# erzeuge künstliche Messdaten
X, y = make_blobs(n_samples=200, centers=2, random_state=0, cluster_std=0.8)

# teile in Trainings- und Testdaten auf
X_train, X_test, y_train, y_test = train_test_split(X,y)

score_training = []
score_testing = []

# FOR-Schleife
for C in [0.1, 1, 10, 100, 1000, 1e10]:
    # Auswahl des Modells 
    model = SVC(kernel='linear', C=C)

    # Training 
    model.fit(X_train, y_train)
   
    # Validierung
    sc_train = model.score(X_train, y_train)
    sc_test  = model.score(X_test, y_test)
   
    score_training.append(sc_train)
    score_testing.append(sc_test)
    
    # Visualisierung
    fig, ax = plt.subplots(figsize=(8,6))
    ax.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn')
    plot_svc_decision_function(model)
    ax.set_title('C = {}'.format(C))

print(score_training)
print(score_testing)

# Übungsaufgaben zur Vertiefung

#### Aufgabe 12.1

Laden Sie den Datensatz mit Brustkrebsdaten, der in Scikit-Learn hinterlegt ist: 
> https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html 

Tipp: funktioniert analog zu der Iris-Datensammlung wie in Part 10.

Führen Sie eine Klassifikation mit einer **linearen SVM** durch, um aus den ersten beiden Features ('mean radius' und 'mean texture') prognostizieren zu können, ob der Tumor bösartig (0) oder gutartig (1) ist. Splitten Sie dazu die Daten in Trainings- und Testdaten, berechnen Sie den Score des Modells und visualisieren Sie am Ende Ihr Ergebnis.

#### Aufgabe 12.2

Laden Sie den Datensatz mit Brustkrebsdaten, der in Scikit-Learn hinterlegt ist: 
> https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html 

Tipp: funktioniert analog zu der Iris-Datensammlung wie in Part 10.

Führen Sie eine Klassifikation mit einer **Kernel-SVM** (radiale Basis-Funktion) durch, um aus den ersten beiden Features ('mean radius' und 'mean texture') prognostizieren zu können, ob der Tumor bösartig (0) oder gutartig (1) ist. Splitten Sie dazu die Daten in Trainings- und Testdaten, berechnen Sie den Score des Modells und visualisieren Sie am Ende Ihr Ergebnis.

#### Aufgabe 12.3 Wiederholung

Wiederholen Sie die Themen Numpy und Pandas (Part 5 und 6).

* Welchen Funktionalitäten bieten die beiden Module?
* Wie viele Dimensionen kann ein Numpy-Array haben?
* Erzeugen Sie ein 1d-Array mit allen geraden Zahlen zwischen 0 und 20.
* Erzeugen Sie ein 3x5-Matrix (2d-Array) gefüllt mit 0.
* Erzeugen Sie ein 3x5-Array mit zufälligen Zahlen zwischen 0 und 100 und summieren Sie entlang der Zeilen.
* Was ist der Unterschied zwischen einem Series-Objekt und einem DataFrame-Objekt?
* Angenommen, Sie haben einen DataFrame mit den Wochentagen Montag, Dienstag, ... als Zeilenindex und den Namen Alics, Bob und Charlie als Spaltenindex. Die Daten sind die Anzahl der Wochenstunden Vorlesung der Person an dem Tag. 
    * Wie greifen Sie auf alle Daten von Bob zu?
    * Wie greifen Sie auf den Inhalt der Zelle von Alice am Mittwoch zu? 